# LangChain Agents Deep Dive

This is a follow-up notebook from [LangChain Agents Deep Dive with GPT 3.5](https://www.youtube.com/watch?v=jSP-gSEyVeI&list=PLIUOU7oqGTLieV9uTIFMm6_4PXg-hlN6F&index=8) video.

In [2]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv('../.env'))

True

In [3]:
from langchain import OpenAI

llm = OpenAI(temperature=0)

In [4]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain(llm=llm)

# Initialize the math tool
math_tool = Tool(
    name="Calculator",
    func=llm_math.run,
    description="Useful for when you need to answer questions about math."
)

tools = [math_tool]

/Users/mufin/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [13]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [9]:
# Alternative ways is to use tool from langchain-hub
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [12]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [14]:
# Initialize the agent
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iteration=3
)

In [16]:
zero_shot_agent("What is the root of x ** 2 + 3 = 12?")



> Entering new AgentExecutor chain...
 I need to solve for x
Action: Calculator
Action Input: x ** 2 + 3 = 12
Observation: Answer: 3.0
Thought: I now know the final answer
Final Answer: The root of x ** 2 + 3 = 12 is 3.0.

> Finished chain.


{'input': 'What is the root of x ** 2 + 3 = 12?',
 'output': 'The root of x ** 2 + 3 = 12 is 3.0.'}

In [17]:
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")



> Entering new AgentExecutor chain...
 I need to figure out how many apples are in the boxes
Action: Calculator
Action Input: 8 * 2.5
Observation: Answer: 20.0
Thought: I need to add the apples Mary has to the apples in the boxes
Action: Calculator
Action Input: 4 + 20.0
Observation: Answer: 24.0
Thought: I now know the final answer
Final Answer: We have 24 apples.

> Finished chain.


{'input': 'if Mary has four apples and Giorgio brings two and a half apple boxes (apple box contains eight apples), how many apples do we have?',
 'output': 'We have 24 apples.'}

In [18]:
# Asking a non-math question
zero_shot_agent("what is the capital of Vietnam?")



> Entering new AgentExecutor chain...
 I should look up the answer
Action: Google
Action Input: "capital of Vietnam"
Observation: Google is not a valid tool, try another one.
Thought: I should look up the answer in an encyclopedia
Action: Encyclopedia
Action Input: "capital of Vietnam"
Observation: Encyclopedia is not a valid tool, try another one.
Thought: I should look up the answer in an online map
Action: Online Map
Action Input: "capital of Vietnam"
Observation: Online Map is not a valid tool, try another one.
Thought: I should look up the answer in an online search engine
Action: Online Search Engine
Action Input: "capital of Vietnam"
Observation: Online Search Engine is not a valid tool, try another one.
Thought: I should look up the answer in a geography book
Action: Geography Book
Action Input: "capital of Vietnam"
Observation: Geography Book is not a valid tool, try another one.
Thought: I should look up the answer in an atlas
Action: Atlas
Action Input: "capital of Vietnam

ValueError: unknown format from LLM: N/A

In [19]:
# Append a new tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

# Initialize the LLM tool
llm_tool = Tool(
    name="Language Model",
    func=llm_chain.run,
    description="Use this tool for general purpose queries and logic",
)

In [20]:
tools.append(llm_tool)

In [21]:
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [22]:
zero_shot_agent("what is the capital of Vietnam?")



> Entering new AgentExecutor chain...
 I need to find out what the capital of Vietnam is
Action: Language Model
Action Input: What is the capital of Vietnam?
Observation: 

Hanoi.
Thought: I now know the final answer
Final Answer: The capital of Vietnam is Hanoi.

> Finished chain.


{'input': 'what is the capital of Vietnam?',
 'output': 'The capital of Vietnam is Hanoi.'}

In [23]:
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
 I need to calculate this expression
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought: I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

In [25]:
(4.5*2.1) ** 2.2

139.94261298333066

## Interacting with SQL

In [26]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [27]:
from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),    
)

In [28]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [29]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

In [30]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
        obs_id=obs[0], 
        stock_ticker=obs[1], 
        price=obs[2],
        date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [31]:
for obs in observations:
    insert_obs(obs)

In [32]:
from langchain.sql_database import SQLDatabase
from langchain.chains import SQLDatabaseChain

db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

/Users/mufin/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain/chains/sql_database/base.py:63: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [33]:
from langchain.agents import Tool

sql_tool = Tool(
    name='Stock DB',
    func=sql_chain.run,
    description="Useful for when you need to answer questions about stocks " \
                "and their prices."
    
)

## Zero-shot Agent

Agent with no memory.

In [34]:
tools = [math_tool, sql_tool]

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description", 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
)

In [36]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [38]:
result = count_tokens(
    zero_shot_agent, 
    "What is the multiplication of the ratio between stock " +
    "prices for 'ABC' and 'XYZ' in January 3rd and the ratio " + 
    "between the same stock prices in January the 4th?"
)



> Entering new AgentExecutor chain...
 I need to compare the stock prices of 'ABC' and 'XYZ' on two different days
Action: Stock DB
Action Input: Stock prices of 'ABC' and 'XYZ' on January 3rd and January 4th

> Entering new SQLDatabaseChain chain...
Stock prices of 'ABC' and 'XYZ' on January 3rd and January 4th
SQLQuery:SELECT stock_ticker, price, date FROM stocks WHERE (stock_ticker = 'ABC' OR stock_ticker = 'XYZ') AND (date = '2023-01-03' OR date = '2023-01-04')
SQLResult: [('ABC', 232.0, '2023-01-03'), ('ABC', 225.0, '2023-01-04'), ('XYZ', 798.0, '2023-01-03'), ('XYZ', 795.0, '2023-01-04')]
Answer:On January 3rd, ABC was 232.0 and XYZ was 798.0. On January 4th, ABC was 225.0 and XYZ was 795.0.
> Finished chain.

Observation: On January 3rd, ABC was 232.0 and XYZ was 798.0. On January 4th, ABC was 225.0 and XYZ was 795.0.
Thought: I need to calculate the ratio between the two stock prices on each day
Action: Calculator
Action Input: 232.0/798.0 and 225.0/795.0
Observation: Answer:

In [39]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.
Stock DB: Useful for when you need to answer questions about stocks and their prices.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator, Stock DB]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


## Conversational React

The agent remembers things we have talked about and can also reason about them and use tools.

In [40]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

In [41]:
conversational_agent = initialize_agent(
    agent='conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

In [42]:
result = count_tokens(
    conversational_agent, 
    "Please provide me the stock prices for ABC on January the 1st"
)



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Stock DB
Action Input: ABC on January the 1st

> Entering new SQLDatabaseChain chain...
ABC on January the 1st
SQLQuery:SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date = '2023-01-01'
SQLResult: [(200.0,)]
Answer:The price of ABC on January the 1st was 200.0.
> Finished chain.

Observation: The price of ABC on January the 1st was 200.0.
Thought: Do I need to use a tool? No
AI: The price of ABC on January the 1st was 200.0. Is there anything else I can help you with?

> Finished chain.
Spent a total of 1847 tokens


In [43]:
print(conversational_agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

In [51]:
result = count_tokens(
    conversational_agent, 
    "What is the multiplication of the ratio between stock " +
    "prices for 'ABC' and 'XYZ' in January 3rd and the ratio " + 
    "between the same stock prices in January the 4th?"
)



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: The multiplication of the ratio between stock prices for 'ABC' and 'XYZ' in January 3rd and the ratio between the same stock prices in January the 4th

ValueError: LLMMathChain._evaluate("
(ABC_Jan3 / XYZ_Jan3) * (ABC_Jan4 / XYZ_Jan4)
") raised error: 'ABC_Jan3'. Please try again with a valid numerical expression

## React Docstore

Agent with interactions with a docstore.

In [52]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

In [53]:
docstore_agent = initialize_agent(
    tools, 
    llm, 
    agent="react-docstore", 
    verbose=True,
    max_iterations=3
)

In [54]:
count_tokens(docstore_agent, "What were Archimedes' last words?")



> Entering new AgentExecutor chain...
Thought: I need to search Archimedes and find his last words.
Action: Search[Archimedes]
Observation: Archimedes of Syracuse (; c. 287 – c. 212 BC) was a Greek mathematician, physicist, engineer, astronomer, and inventor from the ancient city of Syracuse in Sicily. Although few details of his life are known, he is regarded as one of the leading scientists in classical antiquity. Considered the greatest mathematician of ancient history, and one of the greatest of all time, Archimedes anticipated modern calculus and analysis by applying the concept of the infinitely small and the method of exhaustion to derive and rigorously prove a range of geometrical theorems. These include the area of a circle, the surface area and volume of a sphere, the area of an ellipse, the area under a parabola, the volume of a segment of a paraboloid of revolution, the volume of a segment of a hyperboloid of revolution, and the area of a spiral.Archimedes' other mathemat

{'input': "What were Archimedes' last words?",
 'output': '"Do not disturb my circles"'}

In [55]:
print(docstore_agent.agent.llm_chain.prompt.template)



Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United St

## Self Ask with Search

First-choice agent to use when using LLM's to extract information with a search engine. The agent will ask follow-up questions and use the search functionality to get intermediate answers that help it get to the final answer.

In [61]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool

search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='google search'
    )
]

self_ask_with_search = initialize_agent(
    tools,
    llm,
    agent="self-ask-with-search",verbose=True)

In [62]:
count_tokens(self_ask_with_search, "How old is Candace in Phineas and Ferb?")



> Entering new AgentExecutor chain...
 Yes.
Follow up: Who is Candace in Phineas and Ferb?
Intermediate answer: Candace Gertrude Flynn is the deuteragonist, as well as usually the anti-hero of the Disney Channel animated series Phineas and Ferb.
Follow up: How old is Candace Gertrude Flynn?
Intermediate answer: Candace Gertrude Flynn (voiced by Ashley Tisdale) is characterized as Phineas's fifteen-year-old biological sister and Ferb's stepsister. The character's personality is based on Jennifer Grey's character in Ferris Bueller's Day Off.
So the final answer is: 15

> Finished chain.
Spent a total of 1465 tokens


{'input': 'How old is Candace in Phineas and Ferb?', 'output': '15'}

In [57]:
print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin